<a href="https://colab.research.google.com/github/nipun-taneja/amorphous-yolo/blob/main/notebooks/01_baseline_vs_eiou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>